<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 19: Simulating Planetary Dynamics</h1>



<a name='section_19_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L19.0 Overview</h2>

<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_19_1">L19.1 Simulating Planetary Motion</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_19_1">L19.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_19_2">L19.2 Parallel Dynamics</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_19_2">L19.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_19_3">L19.3 3-body Problem</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_19_3">L19.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_19_4">L19.4 N-body Problem</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_19_4">L19.4 Exercises</a></td>
    </tr>
</table>



In [ ]:
#>>>RUN: L19.0-runcell00

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/L19' >> .git/info/sparse-checkout
!git pull origin main

In [ ]:
#>>>RUN: L19.0-runcell01

#you may need to install this package (it already available in Colab)
#!pip install imageio    #https://imageio.readthedocs.io/en/stable/

In [ ]:
#>>>RUN: L19.0-runcell02

import imageio                        #https://imageio.readthedocs.io/en/stable/
from PIL import Image                 #https://pillow.readthedocs.io/en/stable/reference/Image.html

import numpy as np                    #https://numpy.org/doc/stable/
import torch                          #https://pytorch.org/docs/stable/torch.html
import torch.nn as nn                 #https://pytorch.org/docs/stable/nn.html
import matplotlib.pyplot as plt       #https://matplotlib.org/stable/api/pyplot_summary.html#module-matplotlib.pyplot

from scipy.integrate import odeint    #https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html
from scipy.optimize import minimize   #https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
import csv                            #https://docs.python.org/3/library/csv.html
from matplotlib.patches import Circle #https://matplotlib.org/stable/api/_as_gen/matplotlib.patches.Circle.html
from scipy.integrate import solve_ivp #https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html
from IPython.display import Image     #https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html

In [ ]:
#>>>RUN: L19.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_19_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L19.1 Simulating Planetary Motion</h2>  

| [Top](#section_19_0) | [Previous Section](#section_19_0) | [Exercises](#exercises_19_1) | [Next Section](#section_19_2) |

In [ ]:
#>>>RUN: L19.1-runcell01

#Units
Gc=39.478 #AU^3/yr^2/Msun
re=1.0#AU
ve=2*np.pi*re#2pir/yr
Gmod=Gc/re**2

class star:
    #Save the history
    posh = np.array([])
    velh = np.array([])
    poth = np.array([])
    kinh = np.array([])
    soften =  1e-6
    
    def __init__(self,imass,xinit,yinit,vxinit,vyinit):
        self.mass = imass
        self.rpos = np.array([xinit,yinit])
        self.v    = np.array([vxinit,vyinit])
        self.a    = np.array([0.,0.])
        self.u    = 0
        
    def firststep(self,dt):
        self.rpos=self.rpos+0.5*dt*self.v         

    def firststepv(self,dt):
        self.v=self.v   +0.5*dt*self.a 
        self.rpos=self.rpos+dt*self.v
        self.a[0] = 0.
        self.a[1] = 0.
        self.u    = 0.
        
    def step(self,dt):
        #vnew = self.v   +dt*self.a 
        #self.rpos=self.rpos+dt*self.v
        #self.v   =vnew
        self.v    = self.v   +dt*self.a 
        self.rpos = self.rpos+dt*self.v
        self.posh = np.append(self.posh,self.rpos)
        self.velh = np.append(self.velh,self.v)
        self.poth = np.append(self.poth,self.u)
        self.kinh = np.append(self.kinh,0.5*self.mass*(np.dot(self.v,self.v)))
        #reset
        self.a[0] = 0.
        self.a[1] = 0.
        self.u    = 0.
        
    def force(self,istar):
        drv=istar.rpos-self.rpos
        drs=np.dot(drv,drv)+self.soften
        df=Gmod*drv*(drs**(-1.5))
        #if self.a[0] == 0: #something is up
        #    self.a  = (istar.mass)*df
        #else:
        self.a  += (istar.mass)*df
        istar.update(df,self.mass)
        
    def update(self,df,imass):
        self.a += -imass*df    
        
    def update_u(self,iU):
        self.u += iU
        
    def potential(self,istar):
        drv=istar.rpos-self.rpos
        drs=(np.dot(drv,drv))**(-0.5)
        ulocal = -1.*0.5*Gmod*drs*self.mass*istar.mass
        self.u += ulocal
        istar.update_u(ulocal)
        
#now to get things going we are going to simulate a circle
def circle_v(iM,iR):
    #F=mv^2/r = GMm/(2r)^2=>v=sqrt(GM/4r)
    return np.sqrt(Gc*iM/(4*iR))

def sim(dt=0.001,nsteps=10000):
    radius=1#units of AU
    mass=1#units of Solar Mass
    vup=circle_v(mass,radius)
    a1=star(mass, radius,0,0, vup)
    a2=star(mass,-radius,0,0,-vup)
    a1.firststepv(dt)
    a2.firststepv(dt)
    for t in range(nsteps):
        a1.force(a2)
        a1.potential(a2)
        a1.step(dt)
        a2.step(dt)
    x1vals=np.reshape(a1.posh,(len(a1.posh)//2,2))
    x2vals=np.reshape(a2.posh,(len(a2.posh)//2,2))
    
    #Make the orbit plots square so equal units on both axes
    plt.rcParams['figure.figsize'] = (7,7)

    plt.plot(x1vals[:,0],x1vals[:,1])
    plt.plot(x2vals[:,0],x2vals[:,1])
    plt.show()

    #Revert to normal aspect ratio for energy plots
    plt.rcParams['figure.figsize'] = (9,6)


    plt.plot(range(nsteps),a1.poth+a2.poth,label='potential')
    plt.plot(range(nsteps),a1.kinh+a2.kinh,label='kinetic')
    plt.plot(range(nsteps),a1.kinh+a2.kinh+a1.poth+a2.poth,label='Total', c='g')
    plt.legend()
    plt.xlabel('time step')
    plt.ylabel('energy')
    plt.show()

#First a simple simulation
sim(nsteps=10000)

#Now a simulation with a coarse timestep
sim(dt=0.25,nsteps=100)

In [ ]:
#>>>RUN: L19.1-runcell02

def norm(iVal):
    return iVal/np.mean(iVal)

def simNormE():
    dt=0.001 #units of years
    radius=1 #units of AU
    mass=1 #units of Solar Mass
    vup=circle_v(mass,radius)
    a1=star(mass, radius,0,0, vup)
    a2=star(mass,-radius,0,0,-vup)
    nsteps=10000
    a1.firststepv(dt)
    a2.firststepv(dt)
    for t in range(nsteps):
        a1.force(a2)
        a1.potential(a2)
        a1.step(dt)
        a2.step(dt)

    plt.plot(range(nsteps),norm(a1.poth+a2.poth),label='potential')
    plt.plot(range(nsteps),norm(a1.kinh+a2.kinh),label='kinetic')
    plt.plot(range(nsteps),norm(a1.kinh+a2.kinh+a1.poth+a2.poth),label='Total')
    
    #Comment out the 3 lines above and uncomment the one below to look more
    #closely at variations in the total energy.
    #plt.plot(range(nsteps),norm(a1.kinh+a2.kinh+a1.poth+a2.poth),label='Total',c='g')
    
    plt.legend()
    plt.xlabel('time step')
    plt.ylabel('energy')
    plt.show()

simNormE()

In [ ]:
#>>>RUN: L19.1-runcell03

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def simNoPlot(dt=0.001,nsteps=3000):
    radius=1#units of AU
    mass=1#units of Solar Mass
    vup=circle_v(mass,radius)
    a1=star(mass, radius,0,0, vup)
    a2=star(mass,-radius,0,0,-vup)
    a1.firststepv(dt)
    a2.firststepv(dt)
    for t in range(nsteps):
        a1.force(a2)
        a1.potential(a2)
        a1.step(dt)
        a2.step(dt)
    x1vals=np.reshape(a1.posh,(len(a1.posh)//2,2))
    x2vals=np.reshape(a2.posh,(len(a2.posh)//2,2))
    return np.array([x1vals,x2vals])


def makePlot(nbody,coords,ax,fig,images,ymin=-2,ymax=2,xmin=-2,xmax=2):
    # plot and show learning process
    plt.cla()
    ax.set_xlabel('x(AU)', fontsize=24)
    ax.set_ylabel('y(AU)', fontsize=24)
    ax.set_ylim(ymin,ymax)
    ax.set_xlim(xmin,xmax)
    for body in range(nbody):
        #ax.plot(coords[body][-1,0],coords[body][-1,1],'o', color = '#d2eeff', markerfacecolor = '#0077BE')
        ax.plot(np.flip(coords[body][:,0]),np.flip(coords[body][:,1]), 'o-',color = '#d2eeff',markevery=10000, markerfacecolor = '#0077BE',lw=2)  
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)
    

def animate(coords,iN=2,stepsize=50):
    images = []
    fig, ax = plt.subplots(figsize=(7.5,7))
    for step in range(len(coords[0])-110):
        if step % stepsize == 0:
            makePlot(iN,coords[:,step:step+100],ax,fig,images)
    return images


xvals=simNoPlot() #change default nsteps if desired
images=animate(xvals)
imageio.mimsave('data/L19/orbit_1.gif', images, fps=10, loop=0)
Image(open('data/L19/orbit_1.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory

<a name='exercises_19_1'></a>     

| [Top](#section_19_0) | [Restart Section](#section_19_1) | [Next Section](#section_19_2) |

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-19.1.1 </span>

The code in this section simulates the dynamics of a 2-body system using the leap-frog method. Now, repeat the simulation but with the linear version of the <b>Euler step</b> procedure described in L17.2. Edit the function `step` in the class `eulerstar` below and plot the output.

What is going on? What do you observe? Is energy conserved?

A) The orbital radii are fixed and energy is conserved.\
B) The orbital radii oscillate about an average distance, and the average total energy is conserved.\
C) The orbital radii steadily decrease and the total energy increases over time.\
D) The orbital radii steadily decrease and the total energy decreases over time.\
E) The orbital radii steadily increase and the total energy increases over time.\
F) The orbital radii steadily increase and the total energy decreases over time.

In [ ]:
#>>>EXERCISE: L19.1.1

class eulerstar:
    #Save the history
    posh = np.array([])
    velh = np.array([])
    poth = np.array([])
    kinh = np.array([])
    soften =  1e-6

    def __init__(self,imass,xinit,yinit,vxinit,vyinit):
        self.mass = imass
        self.rpos = np.array([xinit,yinit])
        self.v    = np.array([vxinit,vyinit])
        self.a    = np.array([0.,0.])
        self.u    = 0
        
    def firststep(self,dt):
        self.rpos=self.rpos+0.5*dt*self.v        
        
    def firststepv(self,dt):
        self.v=self.v   +0.5*dt*self.a 
        self.rpos=self.rpos+dt*self.v
        self.a[0] = 0.
        self.a[1] = 0.
        self.u    = 0.
                
    def step(self,dt):
        #YOUR CODE HERE
           
    def force(self,istar):
        drv=istar.rpos-self.rpos
        drs=np.dot(drv,drv)+self.soften
        df=Gmod*drv*(drs**(-1.5))
        #if self.a[0] == 0: #something is up
        #    self.a  = (istar.mass)*df
        #else:
        self.a  += (istar.mass)*df
        istar.update(df,self.mass)
        
    def update(self,df,imass):
        self.a += -imass*df    
        
    def update_u(self,iU):
        self.u += iU
        
    def potential(self,istar):
        drv=istar.rpos-self.rpos
        drs=(np.dot(drv,drv))**(-0.5)
        ulocal = -1.*0.5*Gmod*drs*self.mass*istar.mass
        self.u += ulocal
        istar.update_u(ulocal)
        
#now to get things going we are going to simulate a circle
def circle_v(iM,iR):
    #F=mv^2/r = GMm/(2r)^2=>v=sqrt(GM/4r)
    return np.sqrt(Gc*iM/(4*iR))

def eulersim(dt=0.001,nsteps=3000):
    radius=1#units of AU
    mass=1#units of Solar Mass
    vup=circle_v(mass,radius)
    a1=eulerstar(mass, radius,0,0, vup)
    a2=eulerstar(mass,-radius,0,0,-vup)
    for t in range(nsteps):
        a1.force(a2)
        a1.potential(a2)
        a1.step(dt)
        a2.step(dt)
        
    x1vals=np.reshape(a1.posh,(len(a1.posh)//2,2))
    x2vals=np.reshape(a2.posh,(len(a2.posh)//2,2))
    
    #Make the orbit plots square so equal units on both axes
    plt.rcParams['figure.figsize'] = (7,7)

    plt.plot(x1vals[:,0],x1vals[:,1])
    plt.plot(x2vals[:,0],x2vals[:,1])
    plt.show()

    #Revert to normal aspect ratio for energy plots
    plt.rcParams['figure.figsize'] = (9,6)

    plt.plot(range(nsteps),a1.poth+a2.poth,label='potential')
    plt.plot(range(nsteps),a1.kinh+a2.kinh,label='kinetic')
    plt.plot(range(nsteps),a1.kinh+a2.kinh+a1.poth+a2.poth,label='Total')
    plt.legend()
    plt.xlabel('time step')
    plt.ylabel('energy')
    plt.show()
    

#Simulate with default dt and nsteps
eulersim()

#Simulate with course dt and nsteps
#eulersim(dt=0.25,nsteps=100)

#make sim without plots
def eulersimNoPlot(dt=0.001,nsteps=10000):
    radius=1#units of AU
    mass=1#units of Solar Mass
    vup=circle_v(mass,radius)
    a1=eulerstar(mass, radius,0,0, vup)
    a2=eulerstar(mass,-radius,0,0,-vup)
    for t in range(nsteps):
        a1.force(a2)
        a1.potential(a2)
        a1.step(dt)
        a2.step(dt)
    x1vals=np.reshape(a1.posh,(len(a1.posh)//2,2))
    x2vals=np.reshape(a2.posh,(len(a2.posh)//2,2))    
    return np.array([x1vals,x2vals])


#UNCOMMENT TO VIEW ANIMATION

#xvals=eulersimNoPlot()
#images=animate(xvals)
#imageio.mimsave('data/L19/orbit_2.gif', images, fps=10, loop=0)
#Image(open('data/L19/orbit_2.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory

<a name='section_19_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L19.2 Parallel Dynamics</h2>  

| [Top](#section_19_0) | [Previous Section](#section_19_1) | [Exercises](#exercises_19_2) | [Next Section](#section_19_3) |


In [ ]:
#>>>RUN: L19.2-runcell01

x = np.array([[1,2,3]])
y = np.array([[3,4,5]])
print("Vecs x.T,x:\n",x.T,"\n",x)
print()

print("Matrix x.T-x:\n",x.T-x)
print()

print("Vecs y.T,y:\n",y.T,"\n",y)
print()

print("Matrix y.T-y:\n",y.T-y)
print()

dr2 = (x.T-x)**2 + (y.T-y)**2
print("Matrix dr^2:\n",dr2)
print()

mass = np.array([1,2,3])
print("x*m:\n",x * mass)
print()

print("x@m:\n",x @ mass)
print()

print("Matrix (x.T-x) @ mass:\n",(x.T-x) @ mass)

In [ ]:
#>>>RUN: L19.2-runcell02

print("Matrix dr:\n",dr2)
print()

print("Triu dr:\n",np.triu(dr2))
print()

print(" Sum Triu dr:\n",np.sum(np.triu(dr2)))


In [ ]:
#>>>RUN: L19.2-runcell03

#Note, here we make the softening parameter an input,
#whereas in the previous class `star`, the parameter was fixed.
#Now we can adjust it when we call the class.

class stars:
    test=1
    posh = np.array([])
    velh = np.array([])
    toth = np.array([])

    def __init__(self,imass,pos,vpos,n,soften):
        self.mass = imass
        self.rpos = pos
        self.v    = vpos
        self.e    = 0
        self.n    = n
        self.soften    = soften


    def parallelAcc(self): #take in arrays of everything
        xpos=self.rpos[:,0:1]
        ypos=self.rpos[:,1:2]
        dx = xpos.T - xpos
        dy = ypos.T - ypos
        dr2  = dx**2 + dy**2 + self.soften**2
        dr2[dr2>0] = dr2[dr2>0]**(-1.5)
        #idr3 = (dr2**(-1.5))
        ax   = Gmod * (dx*dr2) @ self.mass
        ay   = Gmod * (dy*dr2) @ self.mass
        a = np.vstack((ax,ay))
        self.a = a.T

    def totalE(self):
        xpos=self.rpos[:,0:1]
        ypos=self.rpos[:,1:2]
        dx = xpos.T - xpos
        dy = ypos.T - ypos
        dr = dx**2 + dy**2
        dr = np.sqrt(dx**2 + dy**2)
        dr[dr > 0] = 1./dr[dr > 0]
        idr = -Gmod*self.mass.T*(self.mass*dr)
        totalU = np.sum(np.sum(np.triu(idr)))
        totalK = np.sum(0.5*self.mass*np.sum(self.v**2,1))
        self.e = totalK+totalU

    def firststep(self,dt):
        self.v    = self.v   +0.5*dt*self.a
        self.rpos = self.rpos+dt*self.v

    def step(self,dt):
        self.v    = self.v   +dt*self.a
        self.rpos = self.rpos+dt*self.v
        self.posh = np.append(self.posh,self.rpos)
        self.velh = np.append(self.velh,self.v)
        self.toth = np.append(self.toth,self.e)

#Now let's setup an init script for between 2 and 4 stars
#Give them all the same mass and place them all on a circle of radius
# 1 AU with the velocity appropriate for a 2-body system
def initparts(iN):
    radius=1#units of AU
    mass=1#units of Solar Mass
    vup=circle_v(mass,radius)
    mass = np.ones(iN)*mass # constant mass for now
    #position
    pos=np.array([])
    pos  = np.append(pos,np.array([radius,0]))
    pos  = np.append(pos,np.array([-radius,0]))
    if iN > 2:
        pos  = np.append(pos,np.array([0,radius]))
    if iN > 3:
        pos  = np.append(pos,np.array([0,-radius]))
    #velocity
    vel=np.array([])
    vel  = np.append(vel,np.array([0,vup]))
    vel  = np.append(vel,np.array([0,-vup]))
    if iN > 2:
        vel  = np.append(vel,np.array([-vup,0]))
    if iN > 3:
        vel  = np.append(vel,np.array([ vup,0]))
    pos  = np.reshape(pos,(iN,2))
    vel  = np.reshape(vel,(iN,2))
    return pos,vel,mass

def plotPaths(iN,xvals,evals):
    x1vals=np.reshape(xvals,(len(evals),2*iN))

    #Make the orbit plots square so equal units on both axes
    plt.rcParams['figure.figsize'] = (7,7)

    for i0 in np.arange(iN):
        plt.plot(x1vals[:,2*i0],x1vals[:,2*i0+1])
    plt.show()

    #Revert to normal aspect ratio for energy plots
    plt.rcParams['figure.figsize'] = (9,6)

    plt.plot(range(len(evals)),evals,label='Total')
    plt.legend()
    plt.xlabel('time step')
    plt.ylabel('energy')
    plt.show()


def simN2(iN=2,insteps=5):
    dt=0.001 #units of years
    soften = 1e-6
    pos,vel,mass=initparts(iN)
    allstars = stars(mass,pos,vel,iN,soften)
    allstars.parallelAcc()
    allstars.totalE()
    allstars.firststep(dt)
    for t in range(insteps):
        allstars.parallelAcc()
        allstars.totalE()
        allstars.step(dt)

    plotPaths(iN,allstars.posh,allstars.toth)

simN2(2,insteps=10000)
#simN2(3,insteps=10000)
#simN2(4,insteps=10000)

In [ ]:
#>>>RUN: L19.2-runcell04

#Now let's setup an init script for an unlimited number of stars
def initparts(iN):
    ###Let's simplify the coordinates
    ###sample a radius along the x-axis
    #radius=np.ones(iN)
    #radius[0]=-1
    radius=np.random.uniform(0.5,3,iN)#units of AU
    radius[1]=-radius[0]
    theta=np.random.uniform(0,2.*np.pi,iN)
    #merge positions and make sure each row is planet
    #theta=0 # <<-- uncomment this to unrandomize the angles
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    #now mass
    mass=np.random.uniform(0,1,iN)#units of Solar Mass
    #mass=np.ones(iN)#units of Solar Mass
    #now v
    #v=circle_v(mass,np.abs(radius))*np.random.normal(0,3)
    v=circle_v(mass,np.abs(radius))*np.abs(np.random.normal(0,0.1))
    theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    return pos,v,mass


def simN2(iN=2,insteps=5):
    dt=0.001 #units of years
    pos,vel,mass=initparts(iN)
    soften=1e-6
    allstars = stars(mass,pos,vel,iN,soften)
    allstars.parallelAcc()
    allstars.totalE()
    allstars.firststep(dt)
    for t in range(insteps):
        allstars.parallelAcc()
        allstars.totalE()
        allstars.step(dt)
    plotPaths(iN,allstars.posh,allstars.toth)
    x1vals=np.reshape(allstars.posh,(insteps,iN,2))
    x1vals=np.swapaxes(x1vals,0,1)
    return x1vals

def animate(coords,iN=2):
    images = []
    fig, ax = plt.subplots(figsize=(7.5,7))
    print(len(coords[0]),coords.shape)
    for step in range(len(coords[0])-110):
        if step % 50 == 0:
            makePlot(iN,coords[:,step:step+100],ax,fig,images,ymin=-10,ymax=10,xmin=-10,xmax=10)
    return images

np.random.seed(109)
xvals=simN2(4,insteps=10000)
images=animate(xvals,iN=4)
imageio.mimsave('data/L19/orbit_3.gif', images, fps=10, loop=0)
Image(open('data/L19/orbit_3.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory

<a name='exercises_19_2'></a>     

| [Top](#section_19_0) | [Restart Section](#section_19_2) | [Next Section](#section_19_3) |

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-19.2.1 </span>

At the end of this section, we were increasing the softening parameter to more closely examine the dynamics of the interacting bodies. However, which of the following are indicators that the softening parameter is too LARGE? Select ALL that apply.

A) When the gravitational force between particles is significantly weakened when they are close to each other.

B) When the softening parameter is comparable to or larger than the average distance between particles.

C) When the total energy of the system exhibits a significant drift over time.

D) When the softening parameter is much smaller than any physical scale of interest in the simulation.

E) When the simulation results fail to resolve small-scale structures or close encounters between particles.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-19.2.2 </span>

Repeat the above setup for 100 stars using only 100 steps (to make things a bit faster). Compare the computation time required for the parallel approach, using `simN2` above, vs. a non-parallel approach, using `simN` defined below. How much faster is the parallel approach?

Hint: Use `time.time()` before and after the simulation is run, and print the difference in times.

A) The parallel approach is faster by about a factor of 1000.\
B) The parallel approach is faster by about a factor of 100.\
C) The parallel approach is faster by about a factor of 10.\
D) Both approaches take a comparable amount of time.


In [ ]:
#>>>EXERCISE: L19.2.2

import itertools

def simN(iN,dt=0.001,insteps=1000):
    combos=list(itertools.combinations(np.arange(iN), 2))
    pos,vel,mass=initparts(iN) 
    stars=np.array([])
    for pN in range(iN):
        a = star(mass[pN],pos[pN,0],pos[pN,1],vel[pN,0],vel[pN,1])
        stars = np.append(stars,a)
    for combo in combos:
        stars[combo[0]].force(stars[combo[1]])
    for pStar in stars:
        pStar.firststepv(dt)
    for t in range(insteps):
        for combo in combos:
            stars[combo[0]].force(stars[combo[1]])
            stars[combo[0]].potential(stars[combo[1]])
        for pStar in stars:
            pStar.step(dt)

    totalE=np.zeros(insteps)
    for i0 in np.arange(iN):
        x1vals=np.reshape(stars[i0].posh,(len(stars[i0].posh)//2,2))
        plt.plot(x1vals[:,0],x1vals[:,1])
        totalE += (stars[i0].poth+stars[i0].kinh)
    plt.show()
    plt.plot(range(insteps),totalE,label='Total')
    plt.legend()
    plt.xlabel('time step')
    plt.ylabel('energy')
    plt.show()
    return 


#MAKE A TIME COMPARISON
#YOUR CODE HERE

<a name='section_19_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L19.3 3-body Problem</h2>  

| [Top](#section_19_0) | [Previous Section](#section_19_2) | [Exercises](#exercises_19_3) | [Next Section](#section_19_4) |

In [ ]:
#>>>RUN: L19.3-runcell01

def init3body(iN):
    radius = np.ones(iN)
    #radius=np.random.uniform(0.5,3,iN)#units of AU
    #theta=np.random.uniform(0,2.*np.pi,iN)    
    #merge positions and make sure each row is planet
    theta = np.arange(0,2*np.pi,2*np.pi/iN)
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    #now mass
    #mass=np.random.uniform(0,1,iN)#units of Solar Mass
    mass=np.ones(iN)#units of Solar Mass
    #now v
    v=circle_v(mass,np.abs(radius))*1.5
    theta+=np.pi/2.
    #v=np.zeros(iN)
    #theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    return pos,v,mass



def simN2(iN=3,insteps=5):
    dt=0.001 #units of years
    pos,vel,mass=init3body(iN)
    soften=1e-6
    allstars = stars(mass,pos,vel,iN,soften)
    allstars.parallelAcc()
    allstars.totalE()
    allstars.firststep(dt)
    for t in range(insteps):
        allstars.parallelAcc()
        allstars.totalE()
        allstars.step(dt)
    plotPaths(iN,allstars.posh,allstars.toth)
    x1vals=np.reshape(allstars.posh,(insteps,iN,2))
    x1vals=np.swapaxes(x1vals,0,1)
    print(x1vals.shape)
    return x1vals

def animate(coords,iN=2):
    images = []
    fig, ax = plt.subplots(figsize=(7.5,7))
    print(len(coords[0]),coords.shape)
    for step in range(len(coords[0])-110):
        if step % 50 == 0:
            makePlot(iN,coords[:,step:step+100],ax,fig,images,ymin=-3,ymax=3,xmin=-3,xmax=3)
    return images


xvals=simN2(3,insteps=10000)
images=animate(xvals,iN=3)
imageio.mimsave('data/L19/orbit_4.gif', images, fps=10, loop=0)
Image(open('data/L19/orbit_4.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory

In [ ]:
#>>>RUN: L19.3-runcell02

def init3body(iN):
    radius = np.ones(iN)
    #radius=np.random.uniform(0.5,3,iN)#units of AU
    #theta=np.random.uniform(0,2.*np.pi,iN)    
    #merge positions and make sure each row is planet
    theta = np.arange(0,2*np.pi,2*np.pi/iN)
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    #now mass
    #mass=np.random.uniform(0,1,iN)#units of Solar Mass
    mass=np.ones(iN)#units of Solar Mass
    #now v
    v=circle_v(mass,np.abs(radius))*1.3
    theta+=np.pi/2.
    #v=np.zeros(iN)
    #theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    return pos,v,mass


#np.random.seed(300)
xvals=simN2(3,insteps=10000) #This number of steps may take some time
images=animate(xvals,iN=3)
imageio.mimsave('data/L19/orbit_4_v1.gif', images, fps=10, loop=0)
Image(open('data/L19/orbit_4_v1.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory

In [ ]:
#>>>RUN: L19.3-runcell03

def init3body(iN):
    radius = np.ones(iN)
    #radius=np.random.uniform(0.5,3,iN)#units of AU
    #theta=np.random.uniform(0,2.*np.pi,iN)    
    #merge positions and make sure each row is planet
    theta = np.arange(0,2*np.pi,2*np.pi/iN)
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    #now mass
    #mass=np.random.uniform(0,1,iN)#units of Solar Mass
    mass=np.ones(iN)#units of Solar Mass
    #now v
    v=circle_v(mass,np.abs(radius))*2.0
    theta+=np.pi/2.
    #v=np.zeros(iN)
    #theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    return pos,v,mass

def animate(coords,iN=2,ymin=-10,ymax=10,xmin=-10,xmax=10,stepsize=50):
    images = []
    fig, ax = plt.subplots(figsize=(7.5,7))
    for step in range(len(coords[0])-110):
        if step % stepsize == 0:
            makePlot(iN,coords[:,step:step+100],ax,fig,images,ymin=ymin,ymax=ymax,xmin=xmin,xmax=xmax)
    return images

xvals=simN2(3,insteps=10000) #This number of steps may take some time
images=animate(xvals,iN=3)
imageio.mimsave('data/L19/orbit_4_v2.gif', images, fps=10, loop=0)
Image(open('data/L19/orbit_4_v2.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory

In [ ]:
#>>>RUN: L19.3-runcell04

def init3body(iN,scale):
    radius = np.ones(iN)
    #radius=np.random.uniform(0.5,3,iN)#units of AU
    #theta=np.random.uniform(0,2.*np.pi,iN)
    #merge positions and make sure each row is planet
    theta = np.arange(0,2*np.pi,2*np.pi/iN)
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    #now mass
    #mass=np.random.uniform(0,1,iN)#units of Solar Mass
    mass=np.ones(iN)#units of Solar Mass
    #now v
    v=circle_v(mass,np.abs(radius))*scale
    theta+=np.pi/2.
    #v=np.zeros(iN)
    #theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    return pos,v,mass

def plotJustPaths(iN,xvals,evals):
    x1vals=np.reshape(xvals,(len(evals),2*iN))

    #Make the orbit plots square so equal units on both axes
    plt.rcParams['figure.figsize'] = (7,7)

    for i0 in np.arange(iN):
        plt.plot(x1vals[:,2*i0],x1vals[:,2*i0+1])


def simInit(iN=3,insteps=5,scale=1.0):
    dt=0.001 #units of years
    pos,vel,mass=init3body(iN,scale)
    soften=1e-6
    allstars = stars(mass,pos,vel,iN,soften)
    allstars.parallelAcc()
    allstars.totalE()
    allstars.firststep(dt)
    for t in range(insteps):
        allstars.parallelAcc()
        allstars.totalE()
        allstars.step(dt)
    plotJustPaths(iN,allstars.posh,allstars.toth)
    x1vals=np.reshape(allstars.posh,(insteps,iN,2))
    x1vals=np.swapaxes(x1vals,0,1)
    return x1vals

for scale in np.arange(0.5,2.5,0.1):
    x1vals=simInit(3,5000,scale)

plt.xlim(-10,10)
plt.ylim(-10,10)
plt.xlabel("x[AU]")
plt.ylabel("x[AU]")
plt.show()

#Revert to the standard plot aspect ratio
plt.rcParams['figure.figsize'] = (9,6)

In [ ]:
#>>>RUN: L19.3-runcell05

def init3body(iN,scale):
    radius = np.ones(iN)
    radius[0] = 0.0001
    #radius=np.random.uniform(0.5,3,iN)#units of AU
    #theta=np.random.uniform(0,2.*np.pi,iN)
    #merge positions and make sure each row is planet
    theta = np.arange(0,2*np.pi,2*np.pi/iN)
    theta[0] = 0
    theta[1] = 0
    theta[2] = np.pi*0.5
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    #now mass
    #mass=np.random.uniform(0,1,iN)#units of Solar Mass
    mass=np.ones(iN)#units of Solar Mass
    mass[1] *= 0.01
    mass[2] *= 0.001
    #now v
    mr = np.ones(iN)
    v=circle_v(mr,np.abs(radius*0.25))
    theta+=np.pi/2.
    v[0] = 0
    #v=np.zeros(iN)
    #theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    print(v)
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    print(vcom)
    return pos,v,mass

def plotJustPaths(iN,xvals,evals):
    x1vals=np.reshape(xvals,(len(evals),2*iN))

    #Make the orbit plots square so equal units on both axes
    plt.rcParams['figure.figsize'] = (7,7)

    for i0 in np.arange(iN):
        plt.plot(x1vals[:,2*i0],x1vals[:,2*i0+1])


def simInit(iN=3,insteps=5,scale=1.0):
    dt=0.001 #units of years
    pos,vel,mass=init3body(iN,scale)
    soften=1e-1
    allstars = stars(mass,pos,vel,iN,soften)
    allstars.parallelAcc()
    allstars.totalE()
    allstars.firststep(dt)
    for t in range(insteps):
        allstars.parallelAcc()
        allstars.totalE()
        allstars.step(dt)
    plotJustPaths(iN,allstars.posh,allstars.toth)
    x1vals=np.reshape(allstars.posh,(insteps,iN,2))
    x1vals=np.swapaxes(x1vals,0,1)
    return x1vals

x1vals=simInit(3,5000,1)

plt.xlim(-2,2)
plt.ylim(-2,2)
plt.xlabel("x[AU]")
plt.ylabel("x[AU]")
plt.show()

#Revert to the standard plot aspect ratio
plt.rcParams['figure.figsize'] = (9,6)

images=animate(x1vals,iN=3,ymin=-2,ymax=2,xmin=-2,xmax=2)
imageio.mimsave('data/L19/orbit_4_v3.gif', images, fps=10, loop=0)
Image(open('data/L19/orbit_4_v3.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory

<a name='exercises_19_3'></a>     

| [Top](#section_19_0) | [Restart Section](#section_19_3) | [Next Section](#section_19_4) |

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-19.3.1 </span>

An interesting effect that we could try to simulate is that of <a href="https://en.wikipedia.org/wiki/Orbital_resonance" target="_blank">orbital resonance</a>. Orbital resonances occur when two or more objects orbit a central object such that the periods of their orbits are integer ratios. For instance, a 2:3 resonance occurs when one body completes 2 orbits during the time that it takes the another body to complete 3. Many orbital resonances are unstable and, therefore, we could possibly observe this in our simulation.

Consider the code below, where we want to define the orbital radius of object 2, `radius[2]`, in terms of the orbital radius of object 1, `radius[2]`, and a desired resonance condition `res`. Here, the resonance is defined to be `res=T2/T1`, where `T1` and `T2` are the orbital periods of the objects.

Using Keplers third law, $T^2 \propto a^3$, choose the line below that correctly completes the missing code. Then try running the code for different values of `res`!

A) `radius[2] = radius[1] * (res)**(3/2)`\
B) `radius[2] = radius[1] * (res)**(2/3)`\
C) `radius[2] = radius[1] * (res)`

<br>

In [ ]:
#>>>EXERCISE: L19.3.1

def init3body_res(iN,scale, res):
    radius = np.ones(iN)
    radius[0] = 0.0001

    # Set the semi-major axes for 2:3 resonance
    radius[1] = 1.0  # First planet's semi-major axis (in AU)
    radius[2] = #YOUR CODE HERE  # Second planet's semi-major axis

    #radius=np.random.uniform(0.5,3,iN)#units of AU
    #theta=np.random.uniform(0,2.*np.pi,iN)
    #merge positions and make sure each row is planet
    theta = np.arange(0,2*np.pi,2*np.pi/iN)
    theta[0] = 0
    theta[1] = 0
    theta[2] = np.pi*0.5
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    
    #now mass
    #mass=np.random.uniform(0,1,iN)#units of Solar Mass
    mass=np.ones(iN)#units of Solar Mass
    mass[1] *= 0.001
    mass[2] *= 0.001
    
    #now v
    mr = np.ones(iN)
    v=circle_v(mr,np.abs(radius*0.25)) #np.sqrt(Gc*iM/(4*iR))
    theta+=np.pi/2.
    v[0] = 0
    #v=np.zeros(iN)
    #theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    #print(v)
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    #print(vcom)
    return pos,v,mass

def simInit(iN=3, insteps=5, scale=1.0, res = 1.):
    dt = 0.001  # units of years
    pos, vel, mass = init3body_res(iN, scale, res)
    soften = 1e-1
    allstars = stars(mass, pos, vel, iN, soften)
    allstars.parallelAcc()
    allstars.totalE()
    allstars.firststep(dt)
    for t in range(insteps):
        allstars.parallelAcc()
        allstars.totalE()
        allstars.step(dt)
    plotJustPaths(iN, allstars.posh, allstars.toth)
    x1vals = np.reshape(allstars.posh, (insteps, iN, 2))
    x1vals = np.swapaxes(x1vals, 0, 1)
    return x1vals

x1vals = simInit(3, 5000, 1, 2./3.)

plt.xlim(-2,2)
plt.ylim(-2,2)
plt.xlabel("x[AU]")
plt.ylabel("x[AU]")
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-19.3.2 </span>

In the next section, we will consider N-body simulations, where N is a large number. Compared to 3- or 4-body simulations, what do you think are the complexities or considerations in running an N-body simulation? Select ALL correct options:

A) The computational complexity increases significantly with the number of bodies because of a growth in pairwise interactions.

B) Achieving and maintaining high accuracy becomes challenging as numerical errors can accumulate with the increased number of bodies.

C) The choice of integration method becomes crucial, often requiring more advanced and computationally expensive methods to ensure accuracy.

D) Handling collisions accurately is a consideration, and specialized algorithms are necessary for collision detection and response.

E) Efficient parallelization is essential to distribute the computational workload across multiple processors or GPUs.

F) Determining appropriate initial conditions becomes more challenging with an increased number of interacting bodies.

G) The scalability of the simulation may become an issue, and some algorithms may not efficiently scale to a large number of bodies.

H) N-body simulations demand substantial memory for storing the positions, velocities, and properties of each body, requiring efficient data structures and memory management.

I) Achieving long-term stability is challenging due to accumulated numerical errors, and symplectic integration methods are often preferred for stability.

J) While parallelization can improve performance, it introduces additional overhead, requiring efficient load balancing and communication between parallelized components.

<br>

<a name='section_19_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L19.4 N-body Problem</h2>  

| [Top](#section_19_0) | [Previous Section](#section_19_3) | [Exercises](#exercises_19_4) |


In [ ]:
#>>>RUN: L19.4-runcell01

xarr = np.random.random((10, 1)) * 2 - 1
print(xarr)
print("Max:",np.argmax(xarr))
print("Sort:\n",np.argsort(xarr,axis=0))
xarr=xarr[np.argsort(xarr,axis=0)]
print("half:",xarr[xarr.shape[0]//2])

In [ ]:
#>>>RUN: L19.4-runcell02

class KDTree:
    """Simple KD tree class"""

    # class initialization function
    def __init__(self, data, mins, maxs):
        self.data = np.asarray(data)

        # data should be two-dimensional
        assert self.data.shape[1] == 2

        if mins is None:
            mins = data.min(0)
        if maxs is None:
            maxs = data.max(0)

        self.mins  = np.asarray(mins)
        self.maxs  = np.asarray(maxs)
        self.sizes = self.maxs - self.mins

        self.child1 = None
        self.child2 = None

        if len(data) > 1:
            # sort on the dimension with the largest spread
            largest_dim  = np.argmax(self.sizes)
            i_sort       = np.argsort(self.data[:, largest_dim])
            self.data[:] = self.data[i_sort, :]

            # find split point
            N = self.data.shape[0]
            half_N = int(N / 2)
            split_point = 0.5 * (self.data[half_N, largest_dim] + self.data[half_N - 1, largest_dim])

            # create subnodes
            mins1 = self.mins.copy()
            mins1[largest_dim] = split_point
            maxs2 = self.maxs.copy()
            maxs2[largest_dim] = split_point

            # Recursively build a KD-tree on each sub-node
            self.child1 = KDTree(self.data[half_N:], mins1, self.maxs)
            self.child2 = KDTree(self.data[:half_N], self.mins, maxs2)

    def draw_rectangle(self, ax, depth=None):
        """Recursively plot a visualization of the KD tree region"""
        if depth == 0:
            rect = plt.Rectangle(self.mins, *self.sizes, ec='k', fc='none')
            ax.add_patch(rect)

        if self.child1 is not None:
            if depth is None:
                self.child1.draw_rectangle(ax)
                self.child2.draw_rectangle(ax)
            elif depth > 0:
                self.child1.draw_rectangle(ax, depth - 1)
                self.child2.draw_rectangle(ax, depth - 1)

    
#------------------------------------------------------------
# Create a set of structured random points in two dimensions
np.random.seed(0)

X = np.random.random((30, 2)) * 2 - 1
X[:, 1] *= 0.1
X[:, 1] += X[:, 0] ** 2 #y=x^2

#------------------------------------------------------------
# Use our KD Tree class to recursively divide the space
KDT = KDTree(X, [-1.1, -0.1], [1.1, 1.1])

#------------------------------------------------------------
# Plot four different levels of the KD tree
fig = plt.figure(figsize=(12, 10))
fig.subplots_adjust(wspace=0.2, hspace=0.2,left=0.1, right=0.9, bottom=0.05, top=0.9)

for level in range(1, 5):
    ax = fig.add_subplot(2, 2, level)#, xticks=[], yticks=[])
    ax.scatter(X[:, 0], X[:, 1], s=9)
    KDT.draw_rectangle(ax, depth=level - 1)

    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-0.15, 1.15)
    ax.set_title('level %i' % level)

# suptitle() adds a title to the entire figure
fig.suptitle('$k$d-tree Example')
plt.show()

In [ ]:
#>>>RUN: L19.4-runcell03

class SquareTree:
    """KD Tree aimed"""

    # class initialization function
    def __init__(self,iMins,iMaxs,iAxis=0,idepth=0):
        self.m     = 0
        self.depth = idepth+1
        self.mins=iMins
        self.maxs=iMaxs
        self.dxs = self.maxs - self.mins
        self.mass   = 0
        self.posm   = np.array([0,0])
        self.pos    = np.array([0,0])
        self.child1 = None
        self.child2 = None
        minsR = self.mins.copy()
        maxsL = self.maxs.copy()
        #iterative funciton that defines the splitting
        if self.depth > 6: #6 levels deep
            return
        oAxis=1 #alternate x and y-axis (don't need to BTW)
        if iAxis == 1:
            oAxis = 0
        maxsL[iAxis]=iMaxs[iAxis]-self.dxs[iAxis]*0.5 #Now split the range between left and right in physical half
        minsR[iAxis]=iMins[iAxis]+self.dxs[iAxis]*0.5 # li
        self.child1 = SquareTree(self.mins,maxsL,oAxis,self.depth)
        self.child2 = SquareTree(minsR,self.maxs,oAxis,self.depth)
            
    def draw_rectangle(self, ax, depth=None):
        if depth == 0:
            rect = plt.Rectangle(self.mins, *self.dxs, ec='k', fc='none')
            ax.add_patch(rect)

        if self.child1 is not None:
            if depth is None:
                self.child1.draw_rectangle(ax)
                self.child2.draw_rectangle(ax)
            elif depth > 0:
                self.child1.draw_rectangle(ax, depth - 1)
                self.child2.draw_rectangle(ax, depth - 1)


#now let's make a star object that steps
class body():
    def __init__(self, ipos, ivpos, imass):
        self.rpos = ipos
        self.v    = ivpos
        self.mass = imass
        self.a    = np.array([0,0])
        
    def firststep(self,dt):
        self.v=self.v   +0.5*dt*self.a 
        self.rpos=self.rpos+dt*self.v
        
    def step(self,dt):
        #vnew = self.v   +dt*self.a 
        #self.rpos=self.rpos+dt*self.v
        #self.v   =vnew
        self.v    = self.v   +dt*self.a 
        self.rpos = self.rpos+dt*self.v

        
def grid(iX):
    Xmin = np.min(iX,axis=0)
    Xmax = np.max(iX,axis=0)
    SQT  = SquareTree(Xmin,Xmax,0)
    return SQT

def points(iBodies):
    lXs = np.array([])
    for pBody in iBodies:
        lXs = np.append(lXs,pBody.rpos)
    lXs = np.reshape(lXs,(len(iBodies),2))
    return lXs


#First we will sample 30 stars in 2D space with 2D velocity
np.random.seed(1234)
X = np.random.random((30, 2)) * 2 - 1
V = np.random.random((30, 2)) * 0.1 
#X[:, 1] *= 0.1
#X[:, 1] += X[:, 0] ** 2
bodies = []
for i0 in range(len(X)):
    mass=1. # they will all have a mass of a solar mass
    pBody = body(X[i0],V[i0],mass)
    bodies.append(pBody)

lXs = points(bodies)
SQT = grid(lXs)
fig1 = plt.figure(figsize=(18, 10))
fig1.subplots_adjust(wspace=0.25, hspace=0.2,left=0.1, right=0.9, bottom=0.05, top=0.9)
for level in range(1, 6):
    ax = fig1.add_subplot(2, 3, level)
    ax.scatter(lXs[:, 0], lXs[:, 1], s=9)
    SQT.draw_rectangle(ax, depth=level)
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_title('level %i' % level)
plt.show()


In [ ]:
#>>>RUN: L19.4-runcell04

class SquareTree:
    """KD Tree aimed"""
    # class initialization function
    def __init__(self,iMins,iMaxs,iAxis=0,idepth=0,isoften=1e-1):
        self.m      = 0
        self.depth  = idepth+1
        self.mins   = iMins
        self.maxs   = iMaxs
        self.dxs    = self.maxs - self.mins
        self.mass   = 0
        self.posm   = np.array([0.,0.])
        self.pos    = np.array([0.,0.])
        self.soften = isoften
        self.maxdepth=4
        self.child1 = None
        self.child2 = None
        minsR = self.mins.copy()
        maxsL = self.maxs.copy()
        #iterative funciton that defines the splitting
        if self.depth > self.maxdepth: #6 levels deep
            return
        oAxis=1 #alternate x and y-axis (don’t need to BTW)
        if iAxis == 1:
            oAxis = 0
        maxsL[iAxis]=iMaxs[iAxis]-self.dxs[iAxis]*0.5 #Now split the range between left and right in physical half
        minsR[iAxis]=iMins[iAxis]+self.dxs[iAxis]*0.5 # li
        self.child1 = SquareTree(self.mins,maxsL,oAxis,self.depth,self.soften)
        self.child2 = SquareTree(minsR,self.maxs,oAxis,self.depth,self.soften)
    def addmass(self,mass,pos):
        #compute for whole grid
        self.m    += mass
        self.posm += mass*pos #intermediate value
        self.pos   = self.posm/self.m #mass weighted avg
        #compute it for the children
        if self.child1 == None:
            return
        if pos[0] <= self.child1.maxs[0] and pos[1] <= self.child1.maxs[1]:
            self.child1.addmass(mass,pos)
        else:
            self.child2.addmass(mass,pos)
    def accel(self,pos):
        #compute it for the grid
        if self.m == 0:
            return 0
        #compute the fine grained resolution if distance > x
        if self.dist(pos)/(np.max(self.dxs)**2) < 4 and self.depth < self.maxdepth:
            a1 = self.child1.accel(pos)
            a2 = self.child2.accel(pos)
            return a1+a2
        else:# compute the coarse grain guy
            dx  = self.pos-pos
            dx3 = (np.dot(dx,dx)+self.soften**2)**(-1.5)
            return Gmod*self.m*dx*dx3
    def dist(self,pos):
        #distance to grid center
        dx  = self.pos-pos
        return np.dot(dx,dx)
    def draw_rectangle(self, ax, depth=None):
        if depth == 0:
            rect = plt.Rectangle(self.mins, *self.dxs, ec='k', fc='none')
            ax.add_patch(rect)
        if self.child1 is not None:
            if depth is None:
                self.child1.draw_rectangle(ax)
                self.child2.draw_rectangle(ax)
            elif depth > 0:
                self.child1.draw_rectangle(ax, depth - 1)
                self.child2.draw_rectangle(ax, depth - 1)

class TreeStars:
    def __init__(self,iBodies,iX,iAxis=0,idepth=0,isoften=1e-1):
        Xmin = np.min(iX,axis=0)
        Xmax = np.max(iX,axis=0)
        self.posh   = np.array([])
        self.grid   = SquareTree(Xmin,Xmax,0,idepth=0,isoften=isoften)
        self.bodies = iBodies
        self.nsteps = 0
        self.soften = isoften
    def addMass(self):
        for pBody in self.bodies:
            self.grid.addmass(pBody.mass,pBody.rpos)
    def force(self):
        for pBody in self.bodies:
            pBody.a = self.grid.accel(pBody.rpos)
    def firststep(self,dt):
        for pBody in self.bodies:
            pBody.firststep(dt)
    def step(self,dt):
        for pBody in self.bodies:
            pBody.step(dt)
    def points(self):
        lXs = np.array([])
        for pBody in self.bodies:
            lXs = np.append(lXs,pBody.rpos)
        lXs = np.reshape(lXs,(len(self.bodies),2))
        return lXs
    def store(self):
        lXs = self.points()
        self.posh = np.append(self.posh,lXs)
    def history(self):
        return np.reshape(self.posh,(self.nsteps,len(self.bodies),2))
    def regrid(self):
        lX = self.points()
        Xmin = np.min(lX,axis=0)
        Xmax = np.max(lX,axis=0)
        self.grid  = SquareTree(Xmin,Xmax,0,idepth=0,isoften=self.soften)
        self.addMass()
    def allsteps(self,insteps=5000,dt=0.001):
        nsteps=insteps
        self.nsteps+=nsteps
        self.regrid()
        self.force()
        self.firststep(dt)
        self.regrid()
        for t in range(nsteps):
            if t % 500 == 0:
                print("steps:",t)
            self.force()
            self.step(dt)
            self.regrid()
            self.store()
        return self.points()

def animate(coords,iN=2,ymin=-10,ymax=10,xmin=-10,xmax=10,stepsize=50):
    images = []
    fig, ax = plt.subplots(figsize=(7.5,7))
    for step in range(len(coords[0])-110):
        if step % stepsize == 0:
            makePlot(iN,coords[:,step:step+100],ax,fig,images,ymin=ymin,ymax=ymax,xmin=xmin,xmax=xmax)
    return images

treeStar = TreeStars(bodies,X)
lX=treeStar.allsteps()
fig1 = plt.figure(figsize=(10, 10))
fig1.subplots_adjust(wspace=0.1, hspace=0.15,left=0.1, right=0.9,bottom=0.05, top=0.9)
plt.scatter(X[:, 0], X[:, 1])
plt.scatter(lX[:, 0], lX[:, 1])
ax.set_xlabel("x-position")
ax.set_ylabel("y-position")
ax.set_xlim(-1.2, 1.2)
ax.set_ylim(-0.15, 1.15)
plt.show()

tracks=treeStar.history()
tracks=np.swapaxes(tracks,0,1)
images=animate(tracks,iN=30,xmin=-500,xmax=500,ymin=-500,ymax=500)
imageio.mimsave('data/L19/orbit_n_v0.gif', images, fps=10, loop=0)
Image(open('data/L19/orbit_n_v0.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory

In [ ]:
#>>>RUN: L19.4-runcell05

def circle_v(iM,iR):
    #F=mv^2/r = GMm/(2r)^2=>v=sqrt(GM/4r)
    #Units
    Gc=39.478 #AU^3/yr^2/Msun
    return np.sqrt(Gc*iM/(4*iR))

bodies = []
mass=1 # they will all have a mass of a solar mass
X = np.array([1,0])
X = np.vstack((X, np.array([-1,0]) ))
vc=circle_v(mass,1)
V = np.array([0,-vc])
V = np.vstack((V, np.array([0,vc]) ))
#Xr = np.array([[4,4],[-4,-4]])

bodies=[]
pBody = body(X[0],V[0],mass)
bodies.append(pBody)
pBody = body(X[1],V[1],mass)
bodies.append(pBody)

treeStar = TreeStars(bodies,X)
lX=treeStar.allsteps(insteps=3000,dt=0.001)
fig1 = plt.figure(figsize=(10, 10))
tracks=treeStar.history()
tracks=np.swapaxes(tracks,0,1)
ax.set_xlabel("x-position")
ax.set_ylabel("y-position")
plt.plot(tracks[0,:,0],tracks[0,:,1])
plt.plot(tracks[1,:,0],tracks[1,:,1])
plt.show()


In [ ]:
#>>>RUN: L19.4-runcell06

import time

class stars:
    test=1
    soften = 1e-1
    posh = np.array([])
    velh = np.array([])
    toth = np.array([])

    def __init__(self,imass,pos,vpos,n,isoften=1e-1):
        self.mass = imass
        self.rpos = pos
        self.v    = vpos
        self.e    = 0
        self.n    = n
        self.soften = isoften


    def parallelAcc(self): #take in arrays of everything
        xpos=self.rpos[:,0:1]
        ypos=self.rpos[:,1:2]
        dx = xpos.T - xpos
        dy = ypos.T - ypos
        dr2  = dx**2 + dy**2 + self.soften**2
        dr2[dr2>0] = dr2[dr2>0]**(-1.5)
        #idr3 = (dr2**(-1.5))
        #Units
        Gc=39.478 #AU^3/yr^2/Msun
        re=1.0#AU
        Gmod=Gc/re**2
        ax   = Gmod * (dx*dr2) @ self.mass
        ay   = Gmod * (dy*dr2) @ self.mass
        a = np.vstack((ax,ay))
        self.a = a.T

    def totalE(self):
        xpos=self.rpos[:,0:1]
        ypos=self.rpos[:,1:2]
        dx = xpos.T - xpos
        dy = ypos.T - ypos
        dr = dx**2 + dy**2
        dr = np.sqrt(dx**2 + dy**2)
        dr[dr > 0] = 1./dr[dr > 0]
        idr = -Gmod*self.mass.T*(self.mass*dr)
        totalU = np.sum(np.sum(np.triu(idr)))
        totalK = np.sum(0.5*self.mass*np.sum(self.v**2,1))
        self.e = totalK+totalU

    def firststep(self,dt):
        self.v    = self.v   +0.5*dt*self.a
        self.rpos = self.rpos+dt*self.v

    def step(self,dt):
        self.v    = self.v   +dt*self.a
        self.rpos = self.rpos+dt*self.v
        self.posh = np.append(self.posh,self.rpos)
        self.velh = np.append(self.velh,self.v)
        self.toth = np.append(self.toth,self.e)

        

np.random.seed(0)
nparts=10000
nsteps=2
X = np.random.random((nparts, 2)) * 2 - 1
V = np.random.random((nparts, 2)) * 0.1

bodies = []
for i0 in range(len(X)):
    mass=1. # they will all have a mass of a solar mass
    pBody = body(X[i0],V[i0],mass)
    bodies.append(pBody)


t0=time.perf_counter()

treeStar = TreeStars(bodies,X)
lX=treeStar.allsteps(insteps=nsteps)

t1=time.perf_counter()
print("N-body Tree Delta time:",t1-t0)

#Now lets setup a quick init script only for up to 4 stars
def initN(X,V,M,iN):
    pos=X
    mass=M#np.ones(iN)#units of Solar Mass
    #now v
    v=V
    #v=v.T
    #transform to the com frame
    #vcom=np.dot(mass,v)/np.sum(mass)
    #v-=vcom
    return pos,v,mass


def simTest(iX,iV,iM,iN=nparts,insteps=nsteps,dt=0.001,isoften=1e-1):
    #units of years
    pos,vel,mass=initN(iX,iV,iM,iN)
    allstars = stars(mass,pos,vel,iN,isoften=isoften)
    allstars.parallelAcc()
    allstars.firststep(dt)
    for t in range(insteps):
        allstars.parallelAcc()
        allstars.step(dt)
    x1vals=np.reshape(allstars.posh,(insteps,iN,2))
    x1vals=np.swapaxes(x1vals,0,1)
    return x1vals

mass=np.ones(nparts)
t2=time.perf_counter()
simTest(X,V,mass,nparts)
t3=time.perf_counter()

print("N-body Parallel",t3-t2)

<a name='exercises_19_4'></a>     

| [Top](#section_19_0) | [Restart Section](#section_19_4) |

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-19.4.1 </span>

Now given all of this, let's try to do a realistic simulation using a tree structure, and make a gif of what the motion might look like in the center of the galaxy. In particular, we will try simulating 300 stars (10X as many as in the section above), for 20000 time steps.

First, we'll start with a distribution of stars sampled from a Gaussian (why not!). It is advisable to choose a softening parameter that is about the same size as the minimum distance between the stars at their starting positions. **We need to figure out what this softening parameter would be based on the number of stars we have in our simulation.**

To figure this out, run the code below, which generates random 2D data points from a normal distribution for various sample sizes, calculates the minimum pairwise distance among the points, and then plots the mean and standard deviation of these minimum distances across multiple experiments. We use a scale factor of `n` so that the minimum separation is independent of the sample size we choose.

Based on the data, what value of the softening parameter should we choose? Hint, this should be comparable to the average minimum separation. Enter your answer as a number with precision `1e2`.




<br>

In [ ]:
#>>>EXERCISE: L19.4.1

from scipy.spatial import distance_matrix

# Initialize arrays to store results
x = np.array([])  # Sample size
y = np.array([])  # Mean of minimum separations
yerr = np.array([])  # Standard deviation of minimum separations

# Number of toy experiments (distributions to draw)
ntoys = 100

# Loop over different sample sizes
for i0 in np.arange(2, 1000, 50):
    n = i0
    min_separations = np.array([])  # To store min separations for each toy experiment
    
    # Perform toy experiments
    for _ in np.arange(ntoys):
        # Generate n data points from a standard normal distribution (2D points)
        data = np.random.normal(0, 1, (n, 2))*n*1e3
        
        # Compute the distance matrix (pairwise distances)
        dist_matrix = distance_matrix(data, data)
        
        # Find the minimum non-zero distance (non-diagonal elements)
        min_separation = np.min(dist_matrix[np.triu_indices(n, k=1)])  # k=1 excludes diagonal
        
        # Store the minimum separation
        min_separations = np.append(min_separations, min_separation)
    
    # Store results for this sample size
    x = np.append(x, n)
    y = np.append(y, min_separations.mean())
    yerr = np.append(yerr, min_separations.std())

# Plotting the results
plt.errorbar(x, y, yerr=yerr, fmt='o')
plt.yscale('log')
plt.xlabel('Sample Size (N)')
plt.ylabel('Mean Minimum Separation')
plt.title('Minimum Separation Among Pairs vs. Sample Size')
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-19.4.2 </span>

Now let's go ahead and give the simulation a try. We'll use a random Gaussian distribution, as above, and set the initial velocity to zero. Run the code below, after entering your softening parameter as `soften_param`, and observe the behavior. Which of the following best characterizes the observed behavior of the system:

A) The stars do not really do anything, they remain close to their initial positions.\
B) The stars converge inward uniformly.\
C) Some stars converge inward, and others shoot out at great speed.\
D) The stars begin to coalesce towards areas that are not necessarily at the center, forming some flow.\
E) The stars begin to circulate.


<br>

In [ ]:
#>>>EXERCISE: L19.4.2

####Warning this box takes a few minutes to run
nparts=300
nsteps=20000
soften_param = #YOUR CODE HERE

np.random.seed(1)
R     = np.random.normal(0,1,nparts)*nparts*1e3
theta = np.random.uniform(0,2*np.pi,nparts)
X = np.vstack((R*np.cos(theta),R*np.sin(theta)))
X = X.T
V = np.zeros((nparts,2))
mass=np.abs(np.random.normal(2,0.5,nparts))
bodies = []
for i0 in range(len(X)):
    pBody = body(X[i0],V[i0],mass[i0])
    bodies.append(pBody)


t0=time.perf_counter()
treeStar = TreeStars(bodies,X,isoften=soften_param)
treeStar.allsteps(insteps=nsteps,dt=50.0)
t1=time.perf_counter()
print("N-body Tree Delta time:",t1-t0)
tracks=treeStar.history()
tracks=np.swapaxes(tracks,0,1)
images=animate(tracks,iN=nparts,xmin=-500000,xmax=500000,ymin=-500000,ymax=500000,stepsize=100)
imageio.mimsave('data/L19/orbit_n.gif', images, fps=10, loop=0)
Image(open('data/L19/orbit_n.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory


#Alternatively, try running N-body code using the parallel (non-tree) method
############################################################################
#t0=time.perf_counter()
#lXVals=simTest(X,V,mass,nparts,nsteps,dt=50.0,isoften=soften_param)
#t1=time.perf_counter()
#print("Actual N-body Delta time:",t1-t0)
#images=animate(tracks,iN=nparts,xmin=-500000,xmax=500000,ymin=-500000,ymax=500000,stepsize=100)
#imageio.mimsave('data/L19/orbit_n2.gif', images, fps=10, loop=0)
#Image(open('data/L19/orbit_n2.gif','rb').read())
#NOTE: gifs can be opened in COLAB by double-clicking file in related `data` directory